In [43]:
import fileinput

data = [line.strip() for line in fileinput.input('input.txt')]
print(data[0])

Step R must be finished before step Y can begin.


In [44]:
# Create the graph
from collections import defaultdict

graph = defaultdict(list)

for line in data:
    split = line.split()
    
    graph[split[7]].append(split[1])
    


In [87]:
set_graph = defaultdict(set)
for node in graph:
    set_graph[node] = set(graph[node])
    
for node in set_graph:
    print(node + " : " + str(set_graph[node]))

Y : {'R', 'L', 'F', 'P'}
T : {'N'}
L : {'N', 'C'}
B : {'N', 'W', 'F'}
D : {'Z', 'H', 'T', 'I', 'L', 'G', 'U', 'Q', 'V', 'K', 'X'}
E : {'Z', 'I', 'O', 'W', 'G', 'U', 'D', 'Y', 'Q', 'N', 'P'}
Z : {'M', 'J', 'A', 'N', 'P'}
V : {'Z', 'H', 'A'}
I : {'Z', 'W', 'L', 'S', 'G', 'Q', 'V', 'X'}
J : {'H', 'O', 'W', 'F', 'C', 'P', 'B'}
X : {'Z', 'J', 'S', 'Q', 'Y'}
U : {'Z', 'T', 'O', 'A', 'V', 'K', 'N', 'P'}
Q : {'Z', 'J', 'S', 'U', 'Y'}
S : {'W', 'P', 'B', 'K'}
G : {'M', 'H', 'J', 'O', 'L', 'V', 'N'}
O : {'N', 'F', 'C'}
M : {'F'}
K : {'N', 'R', 'F'}
P : {'N', 'O', 'H', 'K'}
A : {'N', 'O', 'F', 'C'}
W : {'M'}
H : {'O'}


In [61]:
from string import ascii_uppercase

def purge_completed(table, just_removed):
    for entry in table:
        table[entry].discard(just_removed)

# Efficient way: Topological sort. Instead, just go through level by level and see what is free.
result = []
eligible_steps = [char for char in ascii_uppercase if char not in set_graph]

while(eligible_steps):
    # Take the head of the list and add it to the result.
    processing_char = eligible_steps[0]
    result.append(processing_char)
    eligible_steps = eligible_steps[1:]
        
    # Purge this character.
    purge_completed(set_graph, processing_char)

    # Collect everything that is now empty.
    for entry in set_graph:
        if not set_graph[entry]:
            eligible_steps.append(entry)
            set_graph[entry] = set("---")
    
    # Sort the results so the head is the first one we'd pick.
    # It'd be better to use a tree here.
    eligible_steps = sorted(eligible_steps)

In [62]:
print(''.join(result))

CFMNLOAHRKPTWBJSYZVGUQXIDE


In [63]:
import networkx as nx

def solve(lines):
    G = nx.DiGraph()
    for line in lines:
        parts = line.split(" ")
        G.add_edge(parts[1], parts[7])
    print(''.join(nx.lexicographical_topological_sort(G)))
    
solve(data)

CFMNLOAHRKPTWBJSYZVGUQXIDE


In [89]:
# Part 2: Simulation attempt.

def get_finishing_time(letter, current_time):
    return current_time + 60 + ord(entry) - ord('A')

time = 0
result = []
work_to_complete = [char for char in ascii_uppercase if char not in set_graph]
finishing_times = defaultdict(int)
MAX_WORKERS = 5

active_workers = 0

# Set the initial finishing time.
for entry in work_to_complete:
    finishing_times[entry] = get_finishing_time(entry, time)

while(work_to_complete or active_workers != 0):
    # Check for anything that completed in this tick.
    # Free up workers as necessary.
    remove_list = []
    for event in finishing_times:
        if finishing_times[event] == time:
            # Assuming there are no nasty corner cases where two events finish at the same time and I need a special ordering.
            result.append(event)
            print("Finishing " + str(event))
            remove_list.append(entry)
            active_workers -= 1
            
            # This has officially completed, so purge it from the graph.
            purge_completed(set_graph, event)
            
            # Collect everything that is now empty.
            for entry in set_graph:
                if not set_graph[entry]:
                    print("Adding " + str(entry) + " to the eligible queue")
                    work_to_complete.append(entry)
                    set_graph[entry] = set("---")
    # Remove everything that was just completed from the dictionary. Not really necessary anyway.
    for entry in remove_list:
        finishing_times[entry] = None
        
    # Sort the results so the head is the first one we'd pick.
    # It'd be better to use a tree here.
    work_to_complete = sorted(work_to_complete)

    while active_workers < MAX_WORKERS and work_to_complete:
        # Transfer all eligible events over to the worker queue.
        head = work_to_complete[0]
        work_to_complete = work_to_complete[1:]
        active_workers += 1
        finishing_times[head] = get_finishing_time(head, time)
        print("Starting work on " + str(head))
        
    time += 1

Starting work on C
Starting work on F
Starting work on N
Starting work on R
Finishing C
Finishing F
Adding M to the eligible queue
Finishing N
Adding T to the eligible queue
Adding L to the eligible queue
Adding O to the eligible queue
Adding K to the eligible queue
Finishing R
Starting work on K
Starting work on L
Starting work on M
Starting work on O
Starting work on T
Finishing K
Finishing L
Finishing M
Adding W to the eligible queue
Finishing O
Adding P to the eligible queue
Adding A to the eligible queue
Adding H to the eligible queue
Finishing T
Starting work on A
Starting work on H
Starting work on P
Starting work on W
Finishing H
Finishing A
Finishing P
Adding Y to the eligible queue
Finishing W
Adding B to the eligible queue
Starting work on B
Starting work on Y
Finishing B
Adding J to the eligible queue
Adding S to the eligible queue
Finishing Y
Starting work on J
Starting work on S
Finishing J
Adding Z to the eligible queue
Finishing S
Starting work on Z
Finishing Z
Adding V

In [90]:
print(''.join(result))

CFNRKLMOTHAPWBYJSZVGUQXIDE
